In [16]:
import os
import cv2
import numpy as np
import networkx as nx
from PIL import Image
import torch
import torch.nn as nn


In [24]:
import os
import cv2
import numpy as np
import torch

base_dir = '/Users/alyssonamaral/Documents/gcn/images'

dogs_dir = os.path.join(base_dir, 'Dog')
cats_dir = os.path.join(base_dir, 'Cat')

def load_and_resize_images(folder, target_size=(256, 256)):
    images = []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        img = cv2.imread(img_path)
        if img is not None:
            img_resized = cv2.resize(img, target_size)
            img_rgb = cv2.cvtColor(img_resized, cv2.COLOR_BGR2RGB)  # Converter BGR para RGB
            img_tensor = torch.from_numpy(img_rgb).permute(2, 0, 1).float()  # Converter para tensor e permutar dimensões
            images.append(img_tensor)
    return images

dog_images = load_and_resize_images(dogs_dir)
cat_images = load_and_resize_images(cats_dir)

Corrupt JPEG data: 65 extraneous bytes before marker 0xd9
Corrupt JPEG data: 226 extraneous bytes before marker 0xd9
Corrupt JPEG data: 162 extraneous bytes before marker 0xd9
Corrupt JPEG data: 2230 extraneous bytes before marker 0xd9
Corrupt JPEG data: 254 extraneous bytes before marker 0xd9
Corrupt JPEG data: 399 extraneous bytes before marker 0xd9
Corrupt JPEG data: 1403 extraneous bytes before marker 0xd9
Corrupt JPEG data: 214 extraneous bytes before marker 0xd9
Corrupt JPEG data: 1153 extraneous bytes before marker 0xd9
Corrupt JPEG data: 99 extraneous bytes before marker 0xd9
Corrupt JPEG data: 128 extraneous bytes before marker 0xd9
Corrupt JPEG data: 239 extraneous bytes before marker 0xd9


In [25]:
if dog_images:
    first_image = dog_images[0]
    print(f"Formato da primeira imagem: {first_image.shape}")
    print(f"Tipo de dado da imagem: {type(first_image)}")

Formato da primeira imagem: torch.Size([3, 256, 256])
Tipo de dado da imagem: <class 'torch.Tensor'>


In [26]:
def image_to_graph(image):
    height, width, channels = image.shape
    graph = nx.Graph()

    for i in range(height):
        for j in range(width):
            rgb_values = image[i, j]
            graph.add_node((i, j), rgb=rgb_values)

    for i in range(height):
        for j in range(width):
            neighbors = [
                (i-1, j), (i+1, j), (i, j-1), (i, j+1),  
                (i-1, j-1), (i-1, j+1), (i+1, j-1), (i+1, j+1)  
            ]
            for ni, nj in neighbors:
                if 0 <= ni < height and 0 <= nj < width:
                    diff = np.linalg.norm(image[i, j] - image[ni, nj])
                    graph.add_edge((i, j), (ni, nj), weight=diff)
    return graph


In [27]:
class MessagePassingLayer(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(MessagePassingLayer, self).__init__()
        self.message_mlp = nn.Sequential(
            nn.Linear(input_dim * 2, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, input_dim)
        )
    
    def forward(self, graph):
        new_node_features = {}
        
        # Para cada nó, calcula a nova representação com base nos vizinhos
        for node in graph.nodes:
            # Extrai o valor do nó e os vizinhos
            node_feature = graph.nodes[node]["rgb"]
            neighbor_features = [graph.nodes[neighbor]["rgb"] for neighbor in graph.neighbors(node)]
            
            # Calcula a mensagem média recebida dos vizinhos
            if neighbor_features:
                aggregated_message = torch.mean(torch.stack(neighbor_features), dim=0)
            else:
                aggregated_message = torch.zeros_like(node_feature)
            
            # Concatena a característica do nó com a mensagem agregada
            combined = torch.cat([node_feature, aggregated_message])
            new_node_feature = self.message_mlp(combined)
            new_node_features[node] = new_node_feature

        # Atualiza os atributos de cada nó
        for node, new_feature in new_node_features.items():
            graph.nodes[node]["rgb"] = new_feature

        return graph

# Função para rodar T rounds de propagação de mensagem
def run_message_passing(graph, T, input_dim, hidden_dim):
    message_passing_layer = MessagePassingLayer(input_dim, hidden_dim)
    
    # Executa T rounds de propagação de mensagem
    for _ in range(T):
        graph = message_passing_layer(graph)
    
    return graph

In [28]:
# Defina os parâmetros da propagação de mensagem
T = 15  # Número de rounds de propagação de mensagem
input_dim = 3  # Dimensão RGB dos nós
hidden_dim = 64  # Dimensão oculta para a MLP de atualização

# Função para agregar os valores dos nós em um vetor
def aggregate_graph_features(graph):
    node_features = np.array([graph.nodes[node]["rgb"] for node in graph.nodes])
    aggregated_vector = np.mean(node_features, axis=0)  # Agrega usando a média
    return aggregated_vector
    
# Executa T rounds de message passing em cada grafo
dog_vectors = []
for img in dog_images:
    graph = image_to_graph(img)  # Converte a imagem para um grafo
    graph = run_message_passing(graph, T, input_dim, hidden_dim)  # Propagação de mensagem
    dog_vectors.append(aggregate_graph_features(graph))  # Agrega as características em um vetor

cat_vectors = []
for img in cat_images:
    graph = image_to_graph(img)  # Converte a imagem para um grafo
    graph = run_message_passing(graph, T, input_dim, hidden_dim)  # Propagação de mensagem
    cat_vectors.append(aggregate_graph_features(graph))  # Agrega as características em um vetor

# Concatena os vetores e rótulos para treino
X = np.vstack([dog_vectors, cat_vectors])
y = np.array([0] * len(dog_vectors) + [1] * len(cat_vectors))  # 0 = cachorro, 1 = gato
X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.long)


RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x512 and 6x64)

In [ ]:
class SimpleMLP(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(SimpleMLP, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, output_dim)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return self.softmax(x)

# Parâmetros para a MLP
input_dim = X.shape[1]  # Dimensão de entrada baseada no vetor RGB médio (3)
hidden_dim = 64
output_dim = 2  # Classes: cachorro ou gato

# Instancia e treina a MLP
mlp_model = SimpleMLP(input_dim, hidden_dim, output_dim)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(mlp_model.parameters(), lr=0.05)

# Loop de treino
num_epochs = 20
for epoch in range(num_epochs):
    optimizer.zero_grad()
    outputs = mlp_model(X_tensor)
    loss = criterion(outputs, y_tensor)
    loss.backward()
    optimizer.step()
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")
